# Advanced RAG Pipeline

[Reference RAG Link](https://docs.llamaindex.ai/en/latest/examples/retrievers/auto_merging_retriever.html#)

In [1]:
# import utils
import configparser
from pprint import pprint

# Read the INI file
config = configparser.RawConfigParser()
config.read("../app/config.ini")

# Access values from the sections
openai_api_section = config["OpenAI"]

OPENAI_API_KEY = openai_api_section["openai_key"]

import os
import openai

# openai.api_key = OPENAI_API_KEY#utils.get_openai_api_key()
# openai.api_key
import utils

openai.api_key = utils.get_openai_api_key()
import jupyter_black

jupyter_black.load()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input source will be set to __record__.app.query.rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .


In [2]:
# from openai import OpenAI

# client = OpenAI()

# response = client.images.generate(
#     model="dall-e-3",
#     prompt="a white siamese cat",
#     size="1024x1024",
#     quality="standard",
#     n=1,
# )

# image_url = response.data[0].url

In [3]:
#!curl https://www.gutenberg.org/cache/epub/47676/pg47676.txt > ../data/Amours.txt
#!ls ./../data
!head -100 ../data/FastiV2.txt


Translator’s Note: 
Ovid’s numerous references throughout the Fasti to the 
rising and setting of stars and constellations, further 
detailed in the relevant index entries, have been checked 
using a computer-based astronomical program (Redshift 
4) set to Rome in 8AD. The Kalends, Nones, Ides, and 
major Festivals of each month are identified in the 
headings against the relevant days.
9
Book I
Book I:Introduction
I’ll speak of divisions of time throughout the Roman year,
Their origins, and the stars that set beneath the earth and 
rise.
Germanicus Caesar, accept this work, with a calm face,
And direct the voyage of my uncertain vessel:
Not scorning this slight honour, but like a god, 
Receiving with favour the homage I pay you.
Here you’ll revisit the sacred rites in the ancient texts,
And review by what events each day is marked.
And here you’ll find the festivals of your House,
And see your father’s and your grandfather’s name:
The prizes they won, that illustrate the calendar,
Th

In [4]:
!ls ../data

AmoresV2.txt           LostPoemsV2.txt        notes.jsonl
ArsAmatoria.txt        LoversAssistant.txt    notes_validation.jsonl
FastiV2.txt            MetamorphosesV2.txt
HeroidesV2.txt         RemediaAmorisV2.txt


In [5]:
from llama_index import SimpleDirectoryReader


f1 = "./../data/FastiV2.txt"
f2 = "./../data/MetamorphosesV2.txt"
f3 = "./../data/AmoresV2.txt"
f4 = "./../data/LostPoemsV2.txt"
f5 = "./../data/HeroidesV2.txt"
f6 = "./../data/ArsAmatoria.txt"
f7 = "./../data/RemediaAmorisV2.txt"
f8 = "./../data/LoversAssistant.txt"
documents = SimpleDirectoryReader(
    input_files=[f1, f2, f3, f4, f5, f6, f7, f8]
).load_data()

In [6]:
print(type(documents), "\n")
print(len(documents), "\n")
print(type(documents[0]))
print(documents[0])

<class 'list'> 

8 

<class 'llama_index.schema.Document'>
Doc ID: 5472ca85-f63a-4617-9819-acf81adfa381
Text: Translator’s Note:  Ovid’s numerous references throughout the
Fasti to the  rising and setting of stars and constellations, further
detailed in the relevant index entries, have been checked  using a
computer-based astronomical program (Redshift  4) set to Rome in 8AD.
The Kalends, Nones, Ides, and  major Festivals of each month are
identified i...


## Basic RAG pipeline

In [7]:
from llama_index import Document

document = Document(text="\n\n".join([doc.text for doc in documents]))

In [8]:
from llama_index import VectorStoreIndex
from llama_index import ServiceContext
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model="local:BAAI/bge-small-en-v1.5",  # local:BAAI/bge-large-en local:BAAI/bge-small-en-v1.5
)
index = VectorStoreIndex.from_documents([document], service_context=service_context)

In [9]:
query_engine = index.as_query_engine()

In [10]:
# Uncomment: Sanity-check response
response = query_engine.query("Who is Aglauros?")
pprint(str(response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


('Aglauros is the daughter of Cecrops and a descendant of Pandion. She is one '
 'of the Cecropides, who are the daughters of Pandion and considered '
 'Athenians.')


## Evaluation setup using TruLens

In [25]:
eval_questions = [
    "How should I respond to the following statement: Stand up, get involved, build a consensus and make your difference.",
    "What is a good response to the following statement: America was founded on the principle of equal justice under law.",
    "Who said: “Her clear conscience mocked rumour’s mendacity, But we are a mob prone to credit sin.”"
    "Who said: They bear punishment with equanimity who have earned it.",
    "What is a relevant quote to the following statement: `Extremely counterintuitive and extremely true`",
    "Who is Aglauros?",
    "What does Thebes want?",
    "What is the Bœotian city?",
    "What is the context of the quote: 'Resist beginnings; the remedy comes too late when'?",
    "Please explain what this means: They bear punishment with equanimity who have earned it."
    "Find a quote similar to the following: 'Resist beginnings; the remedy comes too late when'?",
    "Who said: 'We're slow to believe what wounds us.'",
    "Who is The Queen of Erebus?",
    "Where is the Who is The Queen of Erebus mentioned?",
]
# You can try your own question:
new_question = "Who said: No one ought to be pronounced happy before his death, and his last obsequies.?"
eval_questions.append(new_question)

In [26]:
from trulens_eval import Tru

tru = Tru()

tru.reset_database()

In [27]:
import sqlite3


def check_and_create_table():
    # Connect to the SQLite database
    conn = sqlite3.connect("your_database.db")
    cursor = conn.cursor()

    # Check if the "last_tweets" table exists
    cursor.execute(
        "SELECT name FROM sqlite_master WHERE type='table' AND name='last_tweets'"
    )
    table_exists = cursor.fetchone()

    if table_exists:
        print("The 'last_tweets' table already exists.")
    else:
        # Create the "last_tweets" table
        cursor.execute("CREATE TABLE last_tweets (id INTEGER PRIMARY KEY, tweet TEXT)")
        print("The 'last_tweets' table has been created.")

    # Close the database connection
    conn.close()


# Call the function to check and create the table
check_and_create_table()

The 'last_tweets' table already exists.


In [28]:
from trulens_eval import Feedback, TruLlama, OpenAI

# OpenAI(api_key=OPENAI_API_KEY)
# help(OpenAI)

In [29]:
from utils import get_prebuilt_trulens_recorder


tru_recorder = get_prebuilt_trulens_recorder(query_engine, app_id="Direct Query Engine")

In [30]:
with tru_recorder as recording:
    for question in eval_questions:
        response = query_engine.query(question)

In [31]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()

,app_id,app_json,type,record_id,input,output,tags,record_json,cost_json,perf_json,ts,Answer Relevance,Groundedness,Context Relevance,Answer Relevance_calls,Groundedness_calls,Context Relevance_calls,latency,total_tokens,total_cost
0,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_9718632c1d009fea238164dc2f045d6a,"""How should I respond to the following stateme...","""You should respond by acknowledging the impor...",-,"{""record_id"": ""record_hash_9718632c1d009fea238...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-22T14:24:30.377085"", ""...",2023-12-22T14:24:32.076828,1.0,0.0,0.0,[{'args': {'prompt': 'How should I respond to ...,"[{'args': {'source': 'If you are a Man, make t...",[{'args': {'prompt': 'How should I respond to ...,1,2153,0.003254
1,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_7ece89b8cd8d13fceb15412d1419379f,"""What is a good response to the following stat...","""The principle of equal justice under law is a...",-,"{""record_id"": ""record_hash_7ece89b8cd8d13fceb1...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-22T14:24:32.165459"", ""...",2023-12-22T14:24:34.724533,1.0,1.0,0.0,[{'args': {'prompt': 'What is a good response ...,[{'args': {'source': 'The gods won’t consider ...,[{'args': {'prompt': 'What is a good response ...,2,2147,0.003263
2,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_5893b51ebb2e04698d78d9fe4f21486a,"""Who said: \u201cHer clear conscience mocked r...","""Ovid said: \u201cHer clear conscience mocked ...",-,"{""record_id"": ""record_hash_5893b51ebb2e04698d7...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-22T14:24:34.842778"", ""...",2023-12-22T14:24:36.036643,0.4,0.5,0.0,[{'args': {'prompt': 'Who said: “Her clear con...,[{'args': {'source': 'I’ve sinned in nothing: ...,[{'args': {'prompt': 'Who said: “Her clear con...,1,2079,0.003131
3,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_6f0b97a828a729d754f22a507145a7fb,"""What is a relevant quote to the following sta...","""\""Things which, singly, are of no avail, when...",-,"{""record_id"": ""record_hash_6f0b97a828a729d754f...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-22T14:24:36.127103"", ""...",2023-12-22T14:24:37.173116,0.8,1.0,0.0,[{'args': {'prompt': 'What is a relevant quote...,[{'args': {'source': 'Envy takes a lofty fligh...,[{'args': {'prompt': 'What is a relevant quote...,1,2138,0.003216
4,Direct Query Engine,"{""app_id"": ""Direct Query Engine"", ""tags"": ""-"",...",RetrieverQueryEngine(llama_index.query_engine....,record_hash_a6b6d89578501a83cbd4f5c79c795524,"""Who is Aglauros?""","""Aglauros is a daughter of Cecrops and a desce...",-,"{""record_id"": ""record_hash_a6b6d89578501a83cbd...","{""n_requests"": 1, ""n_successful_requests"": 1, ...","{""start_time"": ""2023-12-22T14:24:37.265697"", ""...",2023-12-22T14:24:38.890015,1.0,NaN,NaN,"[{'args': {'prompt': 'Who is Aglauros?', 'resp...",NaN,NaN,1,2098,0.003166


In [32]:
for idx, row in records.iterrows():
    pprint(("Input:", row["input"]))
    print()
    pprint(("Output:", row["output"]))
    print()
    print()

('Input:',
 '"How should I respond to the following statement: Stand up, get involved, '
 'build a consensus and make your difference."')

('Output:',
 '"You should respond by acknowledging the importance of taking action, '
 'actively participating, and working towards creating a shared agreement or '
 'understanding. By standing up, getting involved, and building a consensus, '
 'you can make a meaningful impact and contribute to making a difference."')


('Input:',
 '"What is a good response to the following statement: America was founded on '
 'the principle of equal justice under law."')

('Output:',
 '"The principle of equal justice under law is a fundamental aspect of any '
 'democratic society. It ensures that all individuals, regardless of their '
 'background or status, are treated fairly and impartially by the legal '
 'system. This principle is not unique to America, as many countries around '
 'the world strive to uphold it. However, it is important to recognize that '
 't

In [34]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.
Dashboard already running at path:   Network URL: http://192.168.4.62:8501



<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>

## Advanced RAG Pipeline
### 1. Sentence Window Retrieval

In [35]:
from llama_index.llms import OpenAI

llm = OpenAI(model="gpt-3.5-turbo", temperature=0.2)

In [22]:
from utils import build_sentence_window_index

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-large-en",
    save_dir="sentence_index",  # local:BAAI/bge-large-en local:BAAI/bge-small-en-v1.5
)

In [23]:
from utils import get_sentence_window_query_engine

sentence_window_engine = get_sentence_window_query_engine(sentence_index)

In [24]:
window_response = sentence_window_engine.query(
    "Who said: 'If you want to be loved, be lovable.'?"
)
print(str(window_response))

ValueError: shapes (1024,) and (384,) not aligned: 1024 (dim 0) != 384 (dim 0)

In [ ]:
tru.reset_database()

tru_recorder_sentence_window = get_prebuilt_trulens_recorder(
    sentence_window_engine, app_id="Sentence Window Query Engine"
)

In [ ]:
for question in eval_questions:
    with tru_recorder_sentence_window as recording:
        response = sentence_window_engine.query(question)
        pprint(question)
        pprint(str(response))
        print("\n\n")

ERROR:trulens_eval.instruments:Error calling wrapped function retrieve.
ERROR:trulens_eval.instruments:Traceback (most recent call last):
  File "/Users/aus10powell/Documents/Projects/OvidQuotes/venv/lib/python3.10/site-packages/trulens_eval/instruments.py", line 738, in wrapper
    rets, cost = Endpoint.track_all_costs_tally(
  File "/Users/aus10powell/Documents/Projects/OvidQuotes/venv/lib/python3.10/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 377, in track_all_costs_tally
    result, cbs = Endpoint.track_all_costs(
  File "/Users/aus10powell/Documents/Projects/OvidQuotes/venv/lib/python3.10/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 303, in track_all_costs
    return Endpoint._track_costs(thunk, with_endpoints=endpoints)
  File "/Users/aus10powell/Documents/Projects/OvidQuotes/venv/lib/python3.10/site-packages/trulens_eval/feedback/provider/endpoint/base.py", line 480, in _track_costs
    result: T = thunk()
  File "/Users/aus10powe

ValueError: shapes (1024,) and (384,) not aligned: 1024 (dim 0) != 384 (dim 0)

In [ ]:
tru.get_leaderboard(app_ids=[])

In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

### 2. Auto-Merging Retrieval

In [ ]:
from utils import build_automerging_index

automerging_index = build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",  # local:BAAI/bge-large-en local:BAAI/bge-small-en-v1.5
)

In [ ]:
from utils import get_automerging_query_engine

automerging_query_engine = get_automerging_query_engine(
    automerging_index,
)

In [ ]:
auto_merging_response = automerging_query_engine.query("Who is The Queen of Erebus?")
print(str(auto_merging_response))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


There is no information provided in the given context about the Queen of Erebus.


In [ ]:
tru.reset_database()

tru_recorder_automerging = get_prebuilt_trulens_recorder(
    automerging_query_engine, app_id="Automerging Query Engine"
)

In [ ]:
import warnings

warnings.filterwarnings("ignore")
for question in eval_questions:
    with tru_recorder_automerging as recording:
        response = automerging_query_engine.query(question)
        pprint("QUESTION:", question)
        pprint("RESPONSE:", response)
        print("\n\n")

AttributeError: 'str' object has no attribute 'write'

In [ ]:
tru.get_leaderboard(app_ids=[])

,Context Relevance,Answer Relevance,latency,total_cost
app_id,,,,
Automerging Query Engine,0.0,1.0,3.0,0.000692


In [ ]:
# launches on http://localhost:8501/
tru.run_dashboard()

Starting dashboard ...
Config file already exists. Skipping writing process.
Credentials file already exists. Skipping writing process.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://192.168.4.62:8501 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>